<a href="https://colab.research.google.com/github/KlaraDewitte/Machine_Learning_course_UGent_D012554_kaggle/blob/master/Copy_of_Copy_of_Poging4SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
trainset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_train.csv")

testset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_test.csv")

In [0]:
labels = trainset.pop('label')
indices = testset.pop('index')

In [0]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
scaler.fit(trainset)
trainset_scaled = pd.DataFrame(scaler.transform(trainset))
testset_scaled = pd.DataFrame(scaler.transform(testset))


In [0]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.model_selection import cross_val_score

In [0]:
X = trainset_scaled
y = labels

In [0]:
params_grid = {'C': [0.01, 0.1, 1, 10, 100, 1000],
          'gamma': [0.0001, 0.001, 0.01, 0.1],
          'kernel':['linear', 'poly', 'rbf', 'sigmoid'] }


grid_clf = GridSearchCV(SVC(class_weight='balanced'), params_grid)
grid_clf.fit(X, y)

print(grid_clf.best_estimator_)


SVC(C=100, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [0]:
clf=SVC(100, gamma=0.1, kernel='rbf')
clf.fit(X,y)
score_auc = np.mean(cross_val_score(clf,X,y,cv=10,scoring='roc_auc'))
print(score_auc)

0.9616015126565586


In [0]:
clf=SVC(C=100, gamma=0.1, kernel='rbf', probability=True)
clf.fit(X,y)
predictions = clf.predict_proba(testset_scaled)[:,1]
print(predictions)

[7.86464544e-07 8.38596033e-01 3.65014951e-03 ... 9.99994852e-01
 5.00000000e-01 1.45586421e-02]


In [0]:
dfpredictions= pd.DataFrame(({'index': indices, 'label':predictions}))
print(dfpredictions)

       index         label
0          0  7.864645e-07
1          1  8.385960e-01
2          2  3.650150e-03
3          3  9.999999e-01
4          4  1.000000e+00
...      ...           ...
12887  12887  6.167860e-02
12888  12888  9.999992e-01
12889  12889  9.999949e-01
12890  12890  5.000000e-01
12891  12891  1.455864e-02

[12892 rows x 2 columns]


In [0]:
filename = "my_prediction_results_Poging4SVM.csv"

#make sure to not write the Pandas index column (index=False)
dfpredictions.to_csv(filename,index=False)